In [ ]:
import os
import sys
sys.path.append('..')
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from glob import glob
from pycocotools import mask as pymask
from imageio import imsave
from datetime import datetime
import torch
import cv2
import imageio

from abcmodel.lib.datasets.cvat import JOINT_LABELS
from higher_hrnet.models.higher_hrnet import HigherHRNet
from higher_hrnet.lib.predictor import Refine
from higher_hrnet.lib.transforms import InferenceTransform
from higher_hrnet.lib.clustering import SpatialClustering
from abcmodel.lib.utils import read_tarfile
from abcmodel.lib.datasets.cvat import deproject_pixel_to_point, Polygon

In [ ]:
def find_peak(hms, thresh=0.1):
    hms[hms<thresh] = 0
    center = np.pad(hms, [[0,0],[2,2],[2,2]])
    up = np.pad(hms, [[0,0],[0,4],[2,2]])
    down = np.pad(hms, [[0,0],[4,0],[2,2]])
    left = np.pad(hms, [[0,0],[2,2],[0,4]])
    right = np.pad(hms, [[0,0],[2,2],[4,0]])

    peak = (center>up)&(center>down)&(center>left)&(center>right)
    peak = peak[:,2:-2,2:-2]
    return peak*hms

def iou(b1, b2):
    xmin = max(b1[0], b2[0])
    ymin = max(b1[1], b2[1])
    xmax = min(b1[2], b2[2])
    ymax = min(b1[3], b2[3])
    area1 = (b1[2]-b1[0])*(b1[3]-b1[1])
    area2 = (b2[2]-b2[0])*(b2[3]-b2[1])
    inter = max((xmax-xmin),0)*max((ymax-ymin),0)
    union = area1+area2-inter
    return inter/union

def calc_iou(bbox1, bbox2):
    return [[iou(b1, b2) for b1 in bbox2] for b2 in bbox1]

## Load Higher HRNet(for pig instance segmentation/keypoint estimation) 

In [ ]:
seg_key_model = HigherHRNet(num_keypoints=13,
                            num_seg_classes=2,
                            dim_tag=5).eval()
seg_key_model.load_state_dict(torch.load(
    os.path.join('/workspace', 'pig', 'model', 'seg_key_model.pth')))
seg_key_model = seg_key_model.cuda().eval()
seg_key_model = Refine(seg_key_model, JOINT_LABELS, average_tag=True)

inference_transform = InferenceTransform(input_size=480)
clustering = SpatialClustering(threshold=0.05, min_pixels=20, margin=.5)

In [ ]:
def predict(img):
    x, t_inv = inference_transform(img)
    x = x.unsqueeze(0).cuda()
    
    seg_pred, hm_preds, tag_preds = seg_key_model(x)
    hr_hm = hm_preds[1].cpu()
    seed = torch.sigmoid(tag_preds[0, -1]).cpu()
    instance_map = clustering(tag_preds)
    instance_map = instance_map.cpu().squeeze()

    ins_map = instance_map.numpy()
    ins_map = cv2.resize(ins_map, (640,480), interpolation = cv2.INTER_NEAREST)
    seg = seg_pred.softmax(dim=1)[0,1].cpu().numpy()
    hms = hm_preds[1][0].cpu().numpy()
    
    return ins_map, seg, hms

## Load CSV(GT) and annotated Image

In [ ]:
weight_df = pd.read_csv(os.path.join('/workspace/pig/data/images_20211125/annotation/1125_data.csv'))
weight_df.head()

# file_list = sorted(glob('/workspace/pig/data/20211125_annotate/*/PID/*/*.png'))
file_list = sorted(glob(os.path.join('/workspace/pig/data/images_20211125/annotation', '*', '*', '*.png')))
print(len(file_list))

data_list = []

for file in file_list:
    file_names = file.split('/')
    pid = file_names[-2]
    fname = file_names[-1]

    prefix, date, xmin, ymin, xmax, ymax =  fname.split('_')
    fid = prefix+'_'+date

    rid = file_names[-3].split('-')[-1].replace('IDs', '')

    date = datetime.strptime(date, '%Y%m%d%H%M%S')
    date = date.strftime("%Y/%m/%d")
    
    ymax = ymax.replace('.png', '')
    
    sub_df = weight_df.query(f'DAY=="{date}" & ROOM =="{rid}" & ID == "{pid}"')

    if len(sub_df)>0:
        data_list.append([
            date, pid, fid, 
            sub_df.ROOM.iloc[0], 
            None, 
            sub_df.GT.iloc[0], 
            int(xmin),
            int(ymin), 
            int(xmax), 
            int(ymax), 
            fname
        ])

df = pd.DataFrame(
    data_list, columns=([
        'date', 'pigID', 'fileID', 'roomID', 'camera', 'GT', 
        'xmin',  'ymin',  'xmax', 'ymax', 'img_path']))

In [ ]:
plt.hist(df.GT, range=(80,120), bins=20)
plt.show()

### アノテーションとのマッチング確認

In [ ]:
df.fileID.unique().shape

In [ ]:
tar_list = sorted(glob('/workspace/pig/data/tar/*/*/*.tar.gz'))

for fid in sorted(df.fileID.unique()[15:20]):
    _df = df.query(f'fileID == "{fid}"')
    file_path = tar_list[np.argmax([(fid in tf) for tf in tar_list])]

    floor_depth = 2440

    rgb, depth = read_tarfile(file_path)
    depth = cv2.resize(depth, (640, 480))

    ins_map, seg, hms = predict(rgb)
    ins_map = cv2.resize(ins_map, (640, 480), interpolation=cv2.INTER_NEAREST).astype(np.uint8)
    hms = cv2.resize(hms.transpose(1, 2, 0), (640, 480)).transpose(2,0,1)

    bbox1 = [[row.xmin, row.ymin, row.xmax, row.ymax] for i, row in _df.iterrows()]
    bbox2 = []
    for i in np.unique(ins_map):
        if i != 0:
            y, x = np.where(ins_map==i)
            xmin, ymin = np.min([x,y], axis=1)
            xmax, ymax = np.max([x,y], axis=1)
            bbox2.append([xmin,ymin,xmax,ymax])

    y_list, x_list = np.where(np.array(calc_iou(bbox1, bbox2))==1)
    for i1, i2 in zip(y_list, x_list):
        mid = np.unique(ins_map)[i2+1]
        mask = (ins_map==mid).astype(np.uint8)
        rle_mask = pymask.encode(np.asfortranarray(mask))
        x,y,w,h = map(int, pymask.toBbox(rle_mask))

        cropped_rgb = rgb[y:y+h,x:x+w] * mask[y:y+h,x:x+w, np.newaxis]
        cropped_depth = depth[y:y+h,x:x+w] * mask[y:y+h,x:x+w]
        cropped_mask = mask[y:y+h,x:x+w]

        height = np.where(cropped_depth != 0, floor_depth - cropped_depth.astype(int), 0)

        d_mean = cropped_depth.mean()
        tl = deproject_pixel_to_point(pixel=[x, y], depth=d_mean, distorted=True)
        br = deproject_pixel_to_point(pixel=[x+w, y+h], depth=d_mean, distorted=True)

        w, l, _ = np.abs(tl - br)
        mm_squared = w * l * (mask.sum() / np.prod(mask.shape))

        print(f'pigID={file_path.split("/")[-2]}IDs/{_df.iloc[i1].pigID}/{_df.iloc[i1]["img_path"]}')
        print(f'idx={i1}, {str(_df.iloc[i1].GT)}kg, w={int(w)}, l={int(l)}, mm_squared={int(mm_squared)}')
        fig, ax = plt.subplots(1, 3, figsize=(4, 4))
        ax[0].imshow(cropped_rgb)
        ax[1].imshow(cropped_mask)
        ax[2].imshow(cropped_depth)
        ax[0].axes.yaxis.set_visible(False)
        ax[1].axes.yaxis.set_visible(False)
        ax[2].axes.yaxis.set_visible(False)
        plt.show()

### Depthの高さあっているか確認

In [ ]:
a = plt.hist(depth[ins_map==0], range=(2000,2500), bins=50)
print('floor depth: ', a[1][np.argmax(a[0])])

In [ ]:
gt_df = []
save_dir = '/workspace/pig/data/images_20211125/images'

for fid in sorted(df.fileID.unique()):
    _df = df.query('fileID == "{}"'.format(fid))
    file_path = tar_list[np.argmax([(fid in tf) for tf in tar_list])]

    rgb, depth = read_tarfile(file_path)
    depth = cv2.resize(depth, (640,480))

    rgb_path = 'rgb_'+fid + '.jpg'
    depth_path = 'depth_'+fid + '.png'
    cv2.imwrite(os.path.join(save_dir, rgb_path), rgb)
    imageio.imsave(os.path.join(save_dir, depth_path), depth.astype(np.uint16))
    
    ins_map, seg, hms = predict(rgb)
    ins_map = cv2.resize(ins_map, (640,480), interpolation=cv2.INTER_NEAREST).astype(np.uint8)
    hms = cv2.resize(hms.transpose(1,2,0), (640,480)).transpose(2,0,1)

    bbox1 = [[row.xmin, row.ymin, row.xmax, row.ymax] for i, row in _df.iterrows()]

    bbox2 = []
    for i in np.unique(ins_map):
        if i != 0:
            y, x = np.where(ins_map==i)
            xmin, ymin = np.min([x,y], axis=1)
            xmax, ymax = np.max([x,y], axis=1)
            bbox2.append([xmin,ymin,xmax,ymax])
        
    y_list, x_list = np.where(np.array(calc_iou(bbox1, bbox2))==1)
    
    for i1, i2 in zip(y_list, x_list):
        mid = np.unique(ins_map)[i2+1]
        mask = (ins_map==mid).astype(np.uint8)
        rle_mask = pymask.encode(np.asfortranarray(mask))
        assert rle_mask is not None
        x,y,w,h = map(int, pymask.toBbox(rle_mask))
        bbox = [x,y,w,h]

        #cropped_rgb = rgb[y:y+h,x:x+w] * mask[y:y+h,x:x+w, np.newaxis]
        cropped_depth = depth[y:y+h,x:x+w] * mask[y:y+h,x:x+w]
        cropped_mask = mask[y:y+h,x:x+w]
        
        hm_path = 'rgb_'+fid+str(x)+str(y)+str(w)+str(h)+'.npy'
        np.save(os.path.join(save_dir, hm_path), hms[:,y:y+h,x:x+w])

        height = np.where(cropped_depth != 0, floor_depth - cropped_depth.astype(int), 0)

        d_mean = cropped_depth.mean()
        tl = deproject_pixel_to_point(pixel=[x, y], depth=d_mean, distorted=True)
        br = deproject_pixel_to_point(pixel=[x+w, y+h], depth=d_mean, distorted=True)

        w, l, _ = np.abs(tl - br)
        mm_squared = w * l * (mask.sum() / np.prod(mask.shape))

        gt_df.append([
            _df.iloc[i1].GT,
            _df.iloc[i1].roomID,
            'None',
            np.sum(cropped_mask),
            bbox,
            floor_depth,
            Polygon(mask),
            mm_squared,
            floor_depth,
            np.nan,
            d_mean,
            height.mean(),
            height.max(),
            height.std(),
            w,
            l,
            'new', 'rgb_'+_df.iloc[i1].fileID+'.jpg', _df.iloc[i1].pigID,
            np.nan,
            np.nan,
            np.nan,
            str(_df.iloc[i1].pigID)+str(_df.iloc[i1].GT), hm_path
        ])

In [ ]:
gt_df = pd.DataFrame(gt_df, columns=([
    'weight', 'loc', 'posture', 'num_pixels', 'bbox', 'floor_depth', 'polygon', 
    'mm_squared','camera_height', 'id', 'd_mean', 'h_mean', 'h_max', 'h_std', 
    'w', 'l', 'subset', 'filename', 'UID', 'lying', 'sitting', 'standing', 'ID', 'heatmap_filename']))

In [ ]:
gt_df.to_pickle(os.path.join(save_dir, 'df_final.pkl'))

# データの確認

In [ ]:
gt_df = []
save_dir = '/workspace/pig/data/images_20211125/images'

for fid in sorted(df.fileID.unique()):
    _df = df.query('fileID == "{}"'.format(fid))
    file_path = tar_list[np.argmax([(fid in tf) for tf in tar_list])]

    rgb, depth = read_tarfile(file_path)
    depth = cv2.resize(depth, (640,480))

    rgb_path = 'rgb_'+fid + '.jpg'
    depth_path = 'depth_'+fid + '.png'
#     cv2.imwrite(os.path.join(save_dir, rgb_path), rgb)
#     imageio.imsave(os.path.join(save_dir, depth_path), depth.astype(np.uint16))
    
    ins_map, seg, hms = predict(rgb)
    ins_map = cv2.resize(ins_map, (640,480), interpolation=cv2.INTER_NEAREST).astype(np.uint8)
    hms = cv2.resize(hms.transpose(1,2,0), (640,480)).transpose(2,0,1)

    bbox1 = [[row.xmin, row.ymin, row.xmax, row.ymax] for i, row in _df.iterrows()]

    bbox2 = []
    for i in np.unique(ins_map):
        if i != 0:
            y, x = np.where(ins_map==i)
            xmin, ymin = np.min([x,y], axis=1)
            xmax, ymax = np.max([x,y], axis=1)
            bbox2.append([xmin,ymin,xmax,ymax])
        
    y_list, x_list = np.where(np.array(calc_iou(bbox1, bbox2))==1)
    
    for i1, i2 in zip(y_list, x_list):
        mid = np.unique(ins_map)[i2+1]
        mask = (ins_map==mid).astype(np.uint8)
        rle_mask = pymask.encode(np.asfortranarray(mask))
        assert rle_mask is not None
        x,y,w,h = map(int, pymask.toBbox(rle_mask))
        bbox = [x,y,w,h]

        #cropped_rgb = rgb[y:y+h,x:x+w] * mask[y:y+h,x:x+w, np.newaxis]
        cropped_depth = depth[y:y+h,x:x+w] * mask[y:y+h,x:x+w]
        cropped_mask = mask[y:y+h,x:x+w]
        
        hm_path = 'rgb_'+fid+str(x)+str(y)+str(w)+str(h)+'.npy'
#         np.save(os.path.join(save_dir, hm_path), hms[:,y:y+h,x:x+w])

        height = np.where(cropped_depth != 0, floor_depth - cropped_depth.astype(int), 0)

        d_mean = cropped_depth.mean()
        tl = deproject_pixel_to_point(pixel=[x, y], depth=d_mean, distorted=True)
        br = deproject_pixel_to_point(pixel=[x+w, y+h], depth=d_mean, distorted=True)

        w, l, _ = np.abs(tl - br)
        mm_squared = w * l * (mask.sum() / np.prod(mask.shape))

        gt_df.append([
            _df.iloc[i1].GT,
            _df.iloc[i1].roomID,
            'None',
            np.sum(cropped_mask),
            bbox,
            floor_depth,
            Polygon(mask),
            mm_squared,
            floor_depth,
            np.nan,
            d_mean,
            height.mean(),
            height.max(),
            height.std(),
            w,
            l,
            'new', 'rgb_'+_df.iloc[i1].fileID+'.jpg', _df.iloc[i1].pigID,
            np.nan,
            np.nan,
            np.nan,
            str(_df.iloc[i1].pigID)+str(_df.iloc[i1].GT), hm_path
        ])